In [1]:
import ee
from geetools import ui, tools, composite, Satellite, cloud_mask, algorithms, wrapper

In [21]:
Map = ui.Map()
Map.show()

Map(basemap={'max_zoom': 19, 'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map …

In [3]:
l8 = Satellite('LANDSAT_8_SR')

In [4]:
l8col = l8.collection

In [12]:
p = ee.Geometry.Point([-70.72, -41.92])

In [18]:
col = l8col.filterBounds(p).map(cloud_mask.landsat8SR_pixelQA()).filterDate('2017-01-01', '2017-03-01')

In [19]:
col = col.map(wrapper(algorithms.Landsat.brdf_correct, satellite='LANDSAT_8_SR')) 

In [22]:
Map.centerObject(p, zoom=13)

In [20]:
ui.eprint(col.size())

8


In [16]:
vis = {'bands':[l8.bands['nir'], l8.bands['swir1'], l8.bands['red']], 'min':0, 'max':5000}

In [23]:
Map.addImageCollection(col, vis, nametags=['system_date'])

In [24]:
comp = composite.closest_date(col, '2017-02-25', 'B1')

In [25]:
Map.addLayer(comp, vis, 'composite 2017-02-25')

In [26]:
Map.addLayer(comp.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates')

clip to first image (target date)

In [27]:
comp_clip = composite.closest_date(col, '2017-02-25', 'B1', clip_to_first=True)

In [28]:
Map.addLayer(comp_clip, vis, 'composite 2017-02-25 clip to first')

EEException: Image.clip: The area for image clipping must be a geometry, a Feature or a FeatureCollection.